In [34]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from linearmodels.panel import PooledOLS


In [36]:
df=pd.read_csv('Ami.csv')
df


,ID_number,Firm_age,Company_name,Index1,ROA,NoofEmployees,StockTurnover,Debt_Asset_Ratio,CI
0,GB05106531,19,0404 INVESTMENTS LIMITED,1,0.081753,421,5.598462,0.257692,0.723644
1,GB05106531,19,0404 INVESTMENTS LIMITED,2,0.170962,455,7.582365,0.110456,0.721738
2,GB05106531,19,0404 INVESTMENTS LIMITED,3,0.061630,400,5.915761,0.208970,0.985358
3,GB05106531,19,0404 INVESTMENTS LIMITED,4,0.053165,357,6.052291,0.307310,1.091978
4,GB05106531,19,0404 INVESTMENTS LIMITED,5,0.031591,313,5.155195,0.336897,1.166045
...,...,...,...,...,...,...,...,...,...
34045,GB03881244,24,ZYTRONIC PLC,1,0.021487,111,8.141463,0.000000,2.240811
34046,GB03881244,24,ZYTRONIC PLC,2,-0.011703,160,5.437393,0.000000,2.064621
34047,GB03881244,24,ZYTRONIC PLC,3,0.096148,186,6.626236,0.000000,1.302199
34048,GB03881244,24,ZYTRONIC PLC,4,0.123314,195,7.377690,0.000000,1.174596


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 34050 entries, ('GB05106531', 1) to ('GB03881244', 5)
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Firm_age          34050 non-null  int64  
 1   Company_name      34050 non-null  object 
 2   ROA               34050 non-null  float64
 3   NoofEmployees     34050 non-null  int64  
 4   StockTurnover     34050 non-null  float64
 5   Debt_Asset_Ratio  34050 non-null  float64
 6   CI                34050 non-null  float64
dtypes: float64(4), int64(2), object(1)
memory usage: 2.2+ MB


In [37]:
df.set_index(['ID_number', 'Index1'], inplace=True)
df

Firm_age              Company_name       ROA  \
ID_number  Index1                                                 
GB05106531 1             19  0404 INVESTMENTS LIMITED  0.081753   
           2             19  0404 INVESTMENTS LIMITED  0.170962   
           3             19  0404 INVESTMENTS LIMITED  0.061630   
           4             19  0404 INVESTMENTS LIMITED  0.053165   
           5             19  0404 INVESTMENTS LIMITED  0.031591   
...                     ...                       ...       ...   
GB03881244 1             24              ZYTRONIC PLC  0.021487   
           2             24              ZYTRONIC PLC -0.011703   
           3             24              ZYTRONIC PLC  0.096148   
           4             24              ZYTRONIC PLC  0.123314   
           5             24              ZYTRONIC PLC  0.156491   

                   NoofEmployees  StockTurnover  Debt_Asset_Ratio        CI  
ID_number  Index1                                                            
GB05106531 1                 421       5.598462          0.257692  0.723644  
           2                 455       7.582365          0.110456  0.721738  
           3                 400       5.915761          0.208970  0.985358  
           4                 357       6.052291          0.307310  1.091978  
           5                 313       5.155195          0.336897  1.166045  
...                          ...            ...               ...       ...  
GB03881244 1                 111       8.141463          0.000000  2.240811  
           2                 160       5.437393          0.000000  2.064621  
           3                 186       6.626236          0.000000  1.302199  
           4                 195       7.377690          0.000000  1.174596  
           5                 191       7.640854          0.000000  1.143605  

[34050 rows x 7 columns]

In [38]:
dependent = df['ROA']
independent = sm.add_constant(df[['StockTurnover','NoofEmployees', 'Firm_age', 'Debt_Asset_Ratio', 'CI']])


In [39]:
model = PooledOLS(dependent, independent)


In [40]:
pooledOLS_res = model.fit()
print(pooledOLS_res)


                          PooledOLS Estimation Summary                          
Dep. Variable:                    ROA   R-squared:                     7.035e-05
Estimator:                  PooledOLS   R-squared (Between):              0.0004
No. Observations:               34050   R-squared (Within):           -2.827e-06
Date:                Thu, Jun 08 2023   R-squared (Overall):           7.035e-05
Time:                        18:33:42   Log-likelihood                -6.939e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      0.4791
Entities:                        6810   P-value                           0.7922
Avg Obs:                       5.0000   Distribution:                 F(5,34044)
Min Obs:                       5.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             0.4791
                            

In [41]:
Q1 = df['ROA'].quantile(0.25)
Q3 = df['ROA'].quantile(0.75)
IQR = Q3 - Q1

# only keep rows in the dataframe that do not have outlier values.
filtered_df = df[~((df['ROA'] < (Q1 - 1.5 * IQR)) | (df['ROA'] > (Q3 + 1.5 * IQR)))]


Q1 = df['StockTurnover'].quantile(0.25)
Q3 = df['StockTurnover'].quantile(0.75)
IQR = Q3 - Q1

# only keep rows in the dataframe that do not have outlier values.
filtered_df = filtered_df[~((filtered_df['StockTurnover'] < (Q1 - 1.5 * IQR)) | (filtered_df['StockTurnover'] > (Q3 + 1.5 * IQR)))]



In [42]:
filtered_df.info()
filtered_df.columns

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 27303 entries, ('GB05106531', 1) to ('GB03881244', 5)
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Firm_age          27303 non-null  int64  
 1   Company_name      27303 non-null  object 
 2   ROA               27303 non-null  float64
 3   NoofEmployees     27303 non-null  int64  
 4   StockTurnover     27303 non-null  float64
 5   Debt_Asset_Ratio  27303 non-null  float64
 6   CI                27303 non-null  float64
dtypes: float64(4), int64(2), object(1)
memory usage: 1.8+ MB


Index(['Firm_age', 'Company_name', 'ROA', 'NoofEmployees', 'StockTurnover',
       'Debt_Asset_Ratio', 'CI'],
      dtype='object')

In [43]:
filtered_df


Firm_age              Company_name       ROA  \
ID_number  Index1                                                 
GB05106531 1             19  0404 INVESTMENTS LIMITED  0.081753   
           2             19  0404 INVESTMENTS LIMITED  0.170962   
           3             19  0404 INVESTMENTS LIMITED  0.061630   
           4             19  0404 INVESTMENTS LIMITED  0.053165   
           5             19  0404 INVESTMENTS LIMITED  0.031591   
...                     ...                       ...       ...   
GB03881244 1             24              ZYTRONIC PLC  0.021487   
           2             24              ZYTRONIC PLC -0.011703   
           3             24              ZYTRONIC PLC  0.096148   
           4             24              ZYTRONIC PLC  0.123314   
           5             24              ZYTRONIC PLC  0.156491   

                   NoofEmployees  StockTurnover  Debt_Asset_Ratio        CI  
ID_number  Index1                                                            
GB05106531 1                 421       5.598462          0.257692  0.723644  
           2                 455       7.582365          0.110456  0.721738  
           3                 400       5.915761          0.208970  0.985358  
           4                 357       6.052291          0.307310  1.091978  
           5                 313       5.155195          0.336897  1.166045  
...                          ...            ...               ...       ...  
GB03881244 1                 111       8.141463          0.000000  2.240811  
           2                 160       5.437393          0.000000  2.064621  
           3                 186       6.626236          0.000000  1.302199  
           4                 195       7.377690          0.000000  1.174596  
           5                 191       7.640854          0.000000  1.143605  

[27303 rows x 7 columns]

In [44]:
# Specify your dependent and independent variables:
dependent1 = filtered_df['ROA']
independent1 = sm.add_constant(filtered_df[['StockTurnover','NoofEmployees', 'Firm_age', 'Debt_Asset_Ratio', 'CI']])

# Run the OLS model
model1 = sm.OLS(dependent1, independent1)

results1 = model1.fit()

# Print the regression results
print(results1.summary())

                            OLS Regression Results                            
Dep. Variable:                    ROA   R-squared:                       0.029
Model:                            OLS   Adj. R-squared:                  0.029
Method:                 Least Squares   F-statistic:                     161.8
Date:                Thu, 08 Jun 2023   Prob (F-statistic):          4.15e-170
Time:                        18:33:58   Log-Likelihood:                 31404.
No. Observations:               27303   AIC:                        -6.280e+04
Df Residuals:                   27297   BIC:                        -6.275e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0566      0.001  

In [45]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 27303 entries, ('GB05106531', 1) to ('GB03881244', 5)
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Firm_age          27303 non-null  int64  
 1   Company_name      27303 non-null  object 
 2   ROA               27303 non-null  float64
 3   NoofEmployees     27303 non-null  int64  
 4   StockTurnover     27303 non-null  float64
 5   Debt_Asset_Ratio  27303 non-null  float64
 6   CI                27303 non-null  float64
dtypes: float64(4), int64(2), object(1)
memory usage: 1.8+ MB


In [46]:
from linearmodels.panel import PanelOLS


# Specify the model
model_fe = PanelOLS.from_formula('ROA ~ StockTurnover + NoofEmployees + Firm_age + Debt_Asset_Ratio + CI + EntityEffects', data=filtered_df, drop_absorbed=True)

# Fit the model
fe_res = model_fe.fit()

# Print the results
print(fe_res)



                          PanelOLS Estimation Summary                           
Dep. Variable:                    ROA   R-squared:                        0.0433
Estimator:                   PanelOLS   R-squared (Between):             -0.1831
No. Observations:               27303   R-squared (Within):               0.0433
Date:                Thu, Jun 08 2023   R-squared (Overall):              0.1033
Time:                        18:34:09   Log-likelihood                 4.518e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      238.10
Entities:                        6810   P-value                           0.0000
Avg Obs:                       4.0093   Distribution:                 F(4,21027)
Min Obs:                       0.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             238.10
                            

/var/folders/bg/8wt9vs595gg7k7s8_yr_pl_w0000gn/T/ipykernel_86953/967110486.py:8: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

Firm_age

  fe_res = model_fe.fit()


In [48]:
filtered_df.head(10)

Firm_age                 Company_name       ROA  \
ID_number  Index1                                                    
GB05106531 1             19     0404 INVESTMENTS LIMITED  0.081753   
           2             19     0404 INVESTMENTS LIMITED  0.170962   
           3             19     0404 INVESTMENTS LIMITED  0.061630   
           4             19     0404 INVESTMENTS LIMITED  0.053165   
           5             19     0404 INVESTMENTS LIMITED  0.031591   
GB07934188 5             11           10 MOTIVES LIMITED  0.227452   
GB06775455 1             15  20TH CENTURY FRAMES LIMITED -0.070315   
           2             15  20TH CENTURY FRAMES LIMITED  0.243429   
           3             15  20TH CENTURY FRAMES LIMITED -0.102077   
GB06085634 1             16   2K POLYMER SYSTEMS LIMITED  0.072325   

                   NoofEmployees  StockTurnover  Debt_Asset_Ratio        CI  
ID_number  Index1                                                            
GB05106531 1                 421       5.598462          0.257692  0.723644  
           2                 455       7.582365          0.110456  0.721738  
           3                 400       5.915761          0.208970  0.985358  
           4                 357       6.052291          0.307310  1.091978  
           5                 313       5.155195          0.336897  1.166045  
GB07934188 5                   8      16.495050          0.094873  0.764041  
GB06775455 1                   2       0.000000          0.000000  0.837686  
           2                   2       0.000000          0.000000  0.403866  
           3                   2       0.000000          0.000000  0.508235  
GB06085634 1                  94       7.812139          0.116866  0.526607

In [50]:
filtered_df = filtered_df.reset_index()

# Create dummy variables for each company
dummies = pd.get_dummies(filtered_df['ID_number'], drop_first=True)

# Add the dummy variables to the dataframe
filtered_df_dummies = pd.concat([filtered_df, dummies], axis=1)

# Specify the model
X = filtered_df_dummies[['StockTurnover','NoofEmployees', 'Firm_age', 'Debt_Asset_Ratio', 'CI'] + list(dummies.columns)]
X = sm.add_constant(X)  # adding a constant
Y = filtered_df_dummies['ROA']

# Fit the model
model = sm.OLS(Y, X).fit()

# Print the results
print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                    ROA   R-squared:                       0.646
Model:                            OLS   Adj. R-squared:                  0.540
Method:                 Least Squares   F-statistic:                     6.116
Date:                Thu, 08 Jun 2023   Prob (F-statistic):               0.00
Time:                        18:42:10   Log-Likelihood:                 45183.
No. Observations:               27303   AIC:                        -7.781e+04
Df Residuals:                   21027   BIC:                        -2.626e+04
Df Model:                        6275                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                0.0703      0.007  

In [62]:
import scipy.stats as st

# Get the Residual Sum of Squares for the OLS model (restricted model):
ssr_restricted_model = results1.ssr

# Get the Residual Sum of Squares for the Fixed Effects model (unrestricted model):
ssr_unrestricted_model = model.ssr

# Get the degrees of freedom of the OLS model:
k1 = len(results1.params)

# Get the degrees of freedom of the Fixed Effects model:
k2 = len(model.params)

# Get total number of observations:
N = len(filtered_df_dummies)

# Calculate the F statistic:
f_statistic = ((ssr_restricted_model - ssr_unrestricted_model) / ssr_unrestricted_model) * ((N-k2) / (k2-k1))
print('F-statistic for FE model =', f_statistic)

# Calculate the critical value of the F-distribution at alpha = 0.05:
alpha = 0.05
f_critical_value = st.f.ppf(1.0-alpha, (k2-k1), (N-k2))
print('F test critical value at alpha of 0.05 =', f_critical_value)


F-statistic for FE model = 5.846584557488894
F test critical value at alpha of 0.05 = 1.033856809257521


In [51]:
print(filtered_df.columns)


Index(['ID_number', 'Index1', 'Firm_age', 'Company_name', 'ROA',
       'NoofEmployees', 'StockTurnover', 'Debt_Asset_Ratio', 'CI'],
      dtype='object')


In [56]:
filtered_df.head()

,ID_number,Index1,Firm_age,Company_name,ROA,NoofEmployees,StockTurnover,Debt_Asset_Ratio,CI
0,GB05106531,1,19,0404 INVESTMENTS LIMITED,0.081753,421,5.598462,0.257692,0.723644
1,GB05106531,2,19,0404 INVESTMENTS LIMITED,0.170962,455,7.582365,0.110456,0.721738
2,GB05106531,3,19,0404 INVESTMENTS LIMITED,0.061630,400,5.915761,0.208970,0.985358
3,GB05106531,4,19,0404 INVESTMENTS LIMITED,0.053165,357,6.052291,0.307310,1.091978
4,GB05106531,5,19,0404 INVESTMENTS LIMITED,0.031591,313,5.155195,0.336897,1.166045


In [60]:
# Here, I assume 'BvD_ID_number' represents entities and 'Year' represents time.
filtered_df.set_index(['ID_number', 'Index1'], inplace=True)
from linearmodels import RandomEffects

# Run Random Effects model again
model_re = RandomEffects.from_formula('ROA ~ StockTurnover + NoofEmployees + Firm_age + Debt_Asset_Ratio + CI + EntityEffects', data=filtered_df)
re_res = model_re.fit()

print(re_res)



                        RandomEffects Estimation Summary                        
Dep. Variable:                    ROA   R-squared:                        0.1337
Estimator:              RandomEffects   R-squared (Between):              0.3519
No. Observations:               27303   R-squared (Within):               0.0308
Date:                Thu, Jun 08 2023   R-squared (Overall):              0.3219
Time:                        18:49:17   Log-likelihood                 4.157e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      842.82
Entities:                        6272   P-value                           0.0000
Avg Obs:                       4.3532   Distribution:                 F(5,27298)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             842.82
                            

In [50]:
from scipy.stats import chi2

# Run Fixed Effects model
model_fe = PanelOLS.from_formula('ROA ~ Stock_Turnover_x + Debt_to_asset_ratio + CI + EntityEffects', data=filtered_df)
fe_res = model_fe.fit()

# Hausman Test
b = fe_res.params
B = re_res.params
v_b = fe_res.cov
v_B = re_res.cov

df = b[np.abs(b) < 1e8].size
chi2_stat = (b-B).dot(np.linalg.inv(v_b - v_B)).dot(b-B)

pval = chi2(df).sf(chi2_stat)

print(f"chi2_stat: {chi2_stat}, p-value: {pval}")


chi2_stat: 412.7856822166844, p-value: 3.763385316061024e-89


In [51]:
filtered_df.head(30)

,,SK_no,Company_name,Firm_age,ROA,Number_of_employees,Stock_Turnover_x,Debt_to_asset_ratio,CI
BvD_ID_number,Year,,,,,,,,
KINGSGATE HEALTH LIMITED,2021,GB06517067,10890,15,0.272344,2,3.342391,0.010325,0.918595
AUTOSMART INTERNATIONAL LTD,2018,GB01395515,20412,45,0.272285,148,9.521305,0.038658,0.582907
GLOBAL DIGITAL SYSTEMS LIMITED,2017,GB01459108,28990,44,0.272263,50,4.414311,0.010498,0.768196
BREAS MEDICAL LIMITED,2020,GB07190611,32500,13,0.272218,31,6.602331,0.025959,0.685523
SLICK STITCH EMBROIDERY COMPANY LIMITED,2017,GB02627116,13300,32,0.272012,246,0.000000,0.000563,0.587436
FORTH DIMENSION DISPLAYS LIMITED,2018,GB05220480,26110,19,0.271910,27,12.794144,2.121454,0.589331
SHS INTERNATIONAL LTD,2021,GB00112075,10860,113,0.271882,329,15.395381,0.113705,0.588597
THOMPSON VALVES LIMITED,2020,GB02791464,28140,30,0.271878,166,7.708832,0.405289,0.690364
LANTOR (UK) LIMITED,2019,GB04180283,13950,22,0.271835,74,6.707906,0.388293,0.590073
